In [1]:
import pandas as pd
%matplotlib inline
import re
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
import numpy
import torch

In [2]:
data = pd.read_csv("train_500.csv")

In [6]:
data = pd.read_csv("train_10.csv", header=None)

/Users/eshan/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,88193,88194,88195,88196,88197,88198,88199,88200,88201,88202
0,NaN,Unnamed: 0,-0.046627,-0.053661,-0.016606,-0.008460,-0.014638,-0.035218,-0.047178,-0.056229,...,-0.056585,-0.058592,-0.056084,-0.042918,-0.051984,-0.037560,-0.036154,-0.036979,-0.027567,5.0
1,0.0,0,0.002950,0.006664,0.011377,0.016712,0.019554,0.023011,0.026756,0.030403,...,-0.017632,-0.016130,-0.013723,-0.011765,-0.008020,-0.005111,-0.001405,-0.000061,0.003155,3.0
2,1.0,1,0.002385,0.006600,0.008403,0.009870,0.011277,0.013772,0.013141,0.009194,...,-0.102335,-0.099241,-0.095605,-0.090430,-0.084785,-0.081589,-0.076316,-0.067557,-0.060863,5.0
3,2.0,2,-0.076791,-0.093903,-0.088368,-0.084582,-0.057753,-0.058818,-0.017571,-0.017056,...,-0.016336,0.010430,0.015678,0.000484,0.022724,0.066227,0.064346,0.002998,0.046097,7.0
4,3.0,3,-0.010467,-0.024078,-0.018980,-0.007179,0.046339,-0.057930,-0.016421,0.027928,...,0.042591,-0.107476,0.055837,-0.011334,-0.006786,0.040524,0.058583,-0.014025,0.048811,4.0


In [14]:
# Just some re-shaping and dimension finding

N = 500 #Number of samples we are using
print("N:",N)
train = data.iloc[0:N,1:].values
print("Train shape",train.shape)
N_train = train.shape[0] #Same as N
NUM_SAMPLES = train.shape[1]-1 #Number of columns, sans the class label

X_train = train[:,:-1]
y_train = train[:,-1]
y_train = y_train.reshape(N_train,1)

N: 500
Train shape (500, 88201)


In [31]:
SAMPLE_RATE = 22050
song = X_train[1]
song_int = (32768*song).astype(np.int16)
print(song_int.dtype)
print(max(song_int))
print(min(song_int))
ipd.Audio(song_int, rate=SAMPLE_RATE)

int16
14103
-13390


In [32]:
from scipy.io.wavfile import write


for i in range(0, X_train.shape[0]):
    write("WaveFiles/xtrain_"+str(i)+".wav", SAMPLE_RATE, (32768*X_train[i]).astype(np.int16))




In [ ]:
# JUST SOME FOURIER TRANSFORM PARAMETERS
SAMPLE_RATE = 22050
BINS_OCTAVE = 12*2
N_OCTAVES = 7
NUM_BINS = BINS_OCTAVE * N_OCTAVES

In [ ]:
# Given a wav time series, makes a mel spectrogram
# which is a short-time fourier transform with
# frequencies on the mel (log) scale.
def mel_spec(y):
    Q = librosa.cqt(y=y, sr=SAMPLE_RATE, bins_per_octave=BINS_OCTAVE,n_bins=NUM_BINS)
    Q_db = librosa.amplitude_to_db(Q,ref=np.max)
    return Q_db

In [ ]:
# This means that the spectrograms are 168 rows (frequencies)
# By 173 columns (time frames)
song = X_train[0]
test_spec = mel_spec(song)
print(test_spec.shape)
FEATS = test_spec.shape[0]
FRAMES = test_spec.shape[1]

In [ ]:
#This creates a tensor, in which we have N_train examples, each of which is a mel-frequency "image"

tmp_train = np.zeros((N_train,FEATS,FRAMES))
for i in range(N_train):
    tmp_train[i,:,:] = mel_spec(X_train[i])



In [ ]:
#Save the tensor, so that we can load it into other models
np.save('Data/xtrain_spec', tmp_train)
np.save('Data/ytrain_spec', y_train)

In [ ]:
librosa.display.specshow(tmp_train[0], y_axis='hz')